In [1]:
import numpy as np
import scipy.optimize as op
import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
#Test which optimizers return the hessian automatically
mu = np.array([0.1, 3])
variances = np.array([0.3, 2.])**2
corr = 0.3
Sigma = np.diag(variances)
Sigma[0, 1] = Sigma[1, 0] = corr * np.sqrt(variances.prod())
Sigma

array([[0.09, 0.18],
       [0.18, 4.  ]])

In [7]:
# a posterior
def neg_lnpost(x):
    return -ss.multivariate_normal.logpdf(x, mean=mu, cov=Sigma)

In [8]:
result = op.minimize(neg_lnpost, x0=[0,0], method="BFGS")
print(result)

      fun: 1.2798961029077345
 hess_inv: array([[0.09      , 0.18      ],
       [0.18      , 4.00000013]])
      jac: array([0.00000000e+00, 1.49011612e-08])
  message: 'Optimization terminated successfully.'
     nfev: 28
      nit: 5
     njev: 7
   status: 0
  success: True
        x: array([0.09999999, 3.00000002])
